In [1]:
import os
! pip install -U langchain-nomic langchain_community tiktoken langchainhub chromadb langchain langgraph tavily-python gpt4all firecrawl-py PyMuPDF sentence_transformers python-dotenv

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
## index

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.docstore.document import Document

# Load PDF documents
pdf_file_paths = ["./data/Medical Policy FY 23-24.pdf", "./data/Provident fund policy.pdf"]

docs_list = []
for pdf_path in pdf_file_paths:
    loader = PyMuPDFLoader(pdf_path)
    try:
        loaded_docs = loader.load()
        docs_list.extend(loaded_docs)
    except Exception as e:
        print(f"Error loading {pdf_path}: {e}")

# Split documents
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=250, chunk_overlap=0)
doc_splitter = text_splitter.split_documents(docs_list)

# Filter and clean metadata
filtered_doc = []
for doc in doc_splitter:
    if isinstance(doc, Document) and hasattr(doc, 'metadata'):
        clean_metadata = {k: v for k, v in doc.metadata.items() if isinstance(v, (str, int, float, bool))}
        filtered_doc.append(Document(page_content=doc.page_content, metadata=clean_metadata))

# Add to vectorDB
embedding = GPT4AllEmbeddings(model_name="all-MiniLM-L6-v2.gguf2.f16.gguf", gpt4all_kwargs={'allow_download': 'True'})
vectorstore = Chroma.from_documents(
    documents=filtered_doc,
    collection_name="rag-chroma",
    embedding=embedding,
)

retriever = vectorstore.as_retriever()

In [4]:
# local_llm = 'gemma'
local_llm = 'llama3'
# local_llm = 'mistral'

In [44]:
# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.prompts import PromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, \
#     SystemMessagePromptTemplate
# from langchain_community.chat_models import ChatOllama
# from sentence_transformers import SentenceTransformer, util
# from langchain.chains import ConversationChain, ConversationalRetrievalChain
# from langchain.chains.conversation.memory import ConversationSummaryMemory
# from langchain.schema import AIMessage, HumanMessage, SystemMessage
# 
# # Load a pre-trained model for semantic similarity
# model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
# 
# llm = ChatOllama(model=local_llm, temperature=0)
# 
# 
# # Define memory
# 
# # Function to format documents
# def format_docs(docs):
#     return "\n\n".join(doc.page_content for doc in docs)
# 
# 
# # Enhanced Function to check relevance using semantic similarity
# def is_relevant(question, context_docs, threshold=0.3):
#     context_text = format_docs(context_docs)
#     context_chunks = context_text.split('\n\n')
#     question_embedding = model.encode(question, convert_to_tensor=True)
#     max_similarity = 0
#     for chunk in context_chunks:
#         context_embedding = model.encode(chunk, convert_to_tensor=True)
#         similarity = util.pytorch_cos_sim(question_embedding, context_embedding)
#         max_similarity = max(max_similarity, similarity.item())
#     return max_similarity > threshold
# 
# 
# # Function to get the appropriate chain based on relevance
# def get_chain(question, context_docs):
#     if is_relevant(question, context_docs):
#         print("Relevant")
# 
        # general_system_template = """system You are an assistant for question-answering tasks.
        # Use the following context to answer the question. Avoid phrases like "Based on the provided context". 
        # Explain the answer in the end and make a heading with paragraph.
        # Question: {question}
        # Context: {context}
        # Answer: assistant"""
#         # 
#         # general_user_template = "Question:```{question}```"
#         # 
#         # messages = [
#         #     SystemMessagePromptTemplate.from_template(general_system_template),
#         #     HumanMessagePromptTemplate.from_template(general_user_template)
#         # ]
#         # 
#         # qa_prompt = ChatPromptTemplate.from_messages( messages )
# 
#         # prompt = PromptTemplate(input_variables=["question", "context"], template=template)
# 
#         # system_message = SystemMessage(content="Act as a Recruiter for a Big Tech Startup")
#         # human_message = HumanMessagePromptTemplate.from_template("{history}")
#         # 
#         # prompt = ChatPromptTemplate(messages=[system_message, human_message])
# 
#         general_system_template = r""" 
#         Given a specific context, please give a short answer to the question, covering the required advices in general and then provide the names all of relevant(even if it relates a bit) products. 
#         ----
#         {context}
#         ----
#         """
# 
# 
#         general_user_template = "Question:```{question}```"
#         messages = [
#             SystemMessagePromptTemplate.from_template(general_system_template),
#             HumanMessagePromptTemplate.from_template(general_user_template)
#         ]
#         
#         qa_prompt = ChatPromptTemplate.from_messages(messages)
# 
#         return ConversationalRetrievalChain(
#             retriever=retriever,
#             return_source_documents=True,
#             memory=ConversationSummaryMemory(llm=llm),
#             verbose=True,
#             combine_docs_chain={'prompt': qa_prompt}
#         )
# 
# 
# # Main function to run the question-answering process
# def answer_question(question):
#     docs = retriever.invoke(question)
#     rag_chain = get_chain(question, docs)
# 
#     if is_relevant(question, docs):
# 
#         generation = rag_chain.jinvoke({"context": docs, "question": question})
#     else:
#         generation = rag_chain.invoke({"question": question})
# 
#     return generation
# 
# 
# # Example question
# question = "What is the vesting period for employer contributions to the provident fund?"
# answer = answer_question(question)
# print(answer)


Relevant


ValidationError: 2 validation errors for ConversationalRetrievalChain
combine_docs_chain
  Can't instantiate abstract class BaseCombineDocumentsChain without an implementation for abstract methods 'acombine_docs', 'combine_docs' (type=type_error)
question_generator
  field required (type=value_error.missing)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, \
    SystemMessagePromptTemplate
from langchain_community.chat_models import ChatOllama
from sentence_transformers import SentenceTransformer, util
from langchain.chains import ConversationChain, ConversationalRetrievalChain
from langchain.chains.conversation.memory import ConversationSummaryMemory
from langchain.schema import AIMessage, HumanMessage, SystemMessage

# Load a pre-trained model for semantic similarity
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

llm = ChatOllama(model=local_llm, temperature=0)


# Define memory

# Function to format documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Enhanced Function to check relevance using semantic similarity
def is_relevant(question, context_docs, threshold=0.3):
    context_text = format_docs(context_docs)
    context_chunks = context_text.split('\n\n')
    question_embedding = model.encode(question, convert_to_tensor=True)
    max_similarity = 0
    for chunk in context_chunks:
        context_embedding = model.encode(chunk, convert_to_tensor=True)
        similarity = util.pytorch_cos_sim(question_embedding, context_embedding)
        max_similarity = max(max_similarity, similarity.item())
    return max_similarity > threshold


# Function to get the appropriate chain based on relevance
def get_chain(question, context_docs):
    if is_relevant(question, context_docs):
        print("Relevant")
        
        

        general_system_template = r""" 
        Given a specific context, please give a short answer to the question, covering the required advices in general and then provide the names all of relevant(even if it relates a bit) products. 
        ----
        {context}
        ----
        """

        review_template_str = """Your job is to use patient reviews to answer questions about their experience at a hospital. Use the following context to answer questions. Be as detailed as possible, but don't make up any information that's not from the context. If you don't know an answer, say you don't know.{context}
        """

        general_user_template = "Question:```{question}```"
        messages = [
            SystemMessagePromptTemplate.from_template(general_system_template),
            HumanMessagePromptTemplate.from_template(general_user_template)
        ]

        qa_prompt = ChatPromptTemplate.from_messages(messages)

        return ConversationalRetrievalChain(
            retriever=retriever,
            return_source_documents=True,
            memory=ConversationSummaryMemory(llm=llm),
            verbose=True,
            combine_docs_chain={'prompt': qa_prompt}
        )


# Main function to run the question-answering process
def answer_question(question):
    docs = retriever.invoke(question)
    rag_chain = get_chain(question, docs)

    if is_relevant(question, docs):

        generation = rag_chain.jinvoke({"context": docs, "question": question})
    else:
        generation = rag_chain.invoke({"question": question})

    return generation


# Example question
question = "What is the vesting period for employer contributions to the provident fund?"
answer = answer_question(question)
print(answer)
